# Trends-based recommendation
In this notebook we will classify the tweets into trends and these trends will help us decide which hashtags to recommend 

In [ ]:
from files_reader import *
import tensorflow as tf

## Get the data

In [ ]:
tweets_and_trends = []
tweets = []
trends = []

tweets_and_trends += (filesReader.read_file(UK_tweets_file))
# tweets_and_trends += (filesReader.read_file(US_tweets_file))
# tweets_and_trends += (filesReader.read_file(AUS_tweets_file))
# tweets_and_trends += (filesReader.read_file(IR_tweets_file))
# tweets_and_trends += (filesReader.read_file(CAN_tweets_file))

random.shuffle(tweets_and_trends)

tweets, trends = filesReader.split_tweets_and_trends(tweets_and_trends)
print(f"We have {len(tweets)} tweets.")


## Tokenize the text

In [ ]:
from keras.preprocessing.text import Tokenizer

tweets_tokenizer = Tokenizer(oov_token="<OOV>")
tweets_tokenizer.fit_on_texts(tweets)
tweets_word_index = tweets_tokenizer.word_index
tweets_index_word = tweets_tokenizer.index_word

print(f"We have {len(tweets_word_index)} different word")
print(tweets_word_index)

## Create the padded sequences

In [ ]:
from keras.utils import pad_sequences
sequence_length = 20

tweets_sequences = tweets_tokenizer.texts_to_sequences(tweets)
tweets_sequences_padded = pad_sequences(tweets_sequences, padding="post", maxlen=sequence_length)

## Map the trends to numbers

In [ ]:
trends_map = {}

counter = 0

for trend in trends:
    if not (trend in trends_map):
        trends_map[trend] = counter
        counter += 1

no_of_trends = len(trends_map)
inv_trends_map = {v: k for k, v in trends_map.items()}
print(f"We have {no_of_trends} different trends")
print(trends_map)

## Create the trends sequences

In [ ]:
trends_sequences = [trends_map[trend] for trend in trends]
print(trends_sequences)

## Encode the trends

In [ ]:
from keras.utils import to_categorical
import tensorflow as tf

encoded_trends = to_categorical(trends_sequences)
print(encoded_trends.shape)

## Prepare the pre-trained embeddings

In [ ]:
from Embeddings.embeddings_matrix import get_embeddings_matrix

embeddings_index_path = "./Embeddings/embeddings_index_object.pkl"
embeddings_matrix, hits, misses = get_embeddings_matrix(tweets_word_index, embeddings_index_path)

print(f"Hits: {hits}, Misses: {misses}")

## Split the data

In [ ]:
training_split = 0.8
training_tweets_count = int(0.8 * len(tweets_sequences_padded))

In [ ]:
train_data = tweets_sequences_padded[0:training_tweets_count]
train_labels = encoded_trends[0:training_tweets_count]
test_data = tweets_sequences_padded[training_tweets_count:]
test_labels = encoded_trends[training_tweets_count:]

print(f'we have {len(train_data)} tweets for training and {len(test_data)} for testing')

## Build the model

In [ ]:
#hyperparameters
embedding_dimensions = 300
lstm_units = 128
dropout_value = 0.2
conv_filters = 64
conv_kernel_size = 5
dense_layers = 10000

In [ ]:
from keras import initializers

no_of_tweets_words = len(tweets_word_index) + 1

hashtag_recommender = tf.keras.Sequential([
    tf.keras.layers.Embedding(
        no_of_tweets_words,
        embedding_dimensions,
        input_length=sequence_length,
        embeddings_initializer=initializers.Constant(embeddings_matrix),
        trainable=True
    ),
    # tf.keras.layers.Dropout(dropout_value),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_units)),
    tf.keras.layers.Dropout(dropout_value),
    tf.keras.layers.Dense(no_of_trends, activation='softmax')
])

hashtag_recommender.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

hashtag_recommender.summary()

In [ ]:
epochs = 20
hashtag_recommender.fit(train_data, train_labels, epochs=epochs, validation_data=(test_data, test_labels))

In [ ]:
import numpy as np

def predict(tweet, tweet_tokenizer, trends_map, inv_trends_map, pad_length, model):
    tweet_sequence = tweet_tokenizer.texts_to_sequences([tweet])[0]
    padded_tweet_sequence = pad_sequences([tweet_sequence], maxlen=pad_length, padding='post')
    prediction = (model.predict(padded_tweet_sequence))
    trends_indices = np.argsort(prediction, axis=-1)[0][-3:]
    return [inv_trends_map[trend_index] for trend_index in trends_indices]

In [ ]:
tweet = "this is so sad what happend to arsenal."

print(predict(tweet, tweets_tokenizer, trends_map, inv_trends_map, sequence_length, hashtag_recommender))